<font size="10">Learning Action-Angle Coordinate Transformations</font>

<font size="4">This notebook loads the trained autoencoder neural network that provides a transformation to action-angle coordinates for the Kepler problem. The Kepler problem is a second order differential equation, given by</font>

$\ddot{r} = \frac{1}{r^3} - \frac{1}{r^2},$

<font size="4">where $r = r(t) \geq 0$ denotes the distance between to bodies interacting through a central force that varies as the inverse square of the distance between them (Newtonian gravity). Data and network training is performed by the notebook Kepler_train.ipynb, found in the action_angle_scripts folder.</font>

<font size="4">This code was written by Jason J. Bramburger and [Bethany Lusch](https://bethanyl.github.io/) to accompany the work in Section 6.3 of the book <i>Data-Driven Methods for Dynamic Systems</i>. We use Tensorflow 2.0 to build the neural network.